# Ensemble: Boosting

- 다양한 모델을 결합하여 예측 성능을 향상시키는 방법
    - 깊이가 얕은 결정트리를 사용해 이전 트리의 오차를 보정하는 방식
    - 순차적으로 경사하강법을 사용해 이전 트리의 오차를 줄여나감
        - 분류 모델에서는 손실함수 LogLoss를 사용해 오차를 줄임
        - 회귀 모델에서는 손실함수 MSE를 사용해 오차를 줄임
    - Boosting 계열은 일반적으로 결정트리 개수를 늘려도 과적합에 강함
    - 대표 알고리즘(모델): GradientBoosting, HistGradientBoosting,  **XGBoost (DMLC)**,  LightGBM(MS), CatBoost

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns  

## GradientBoosting 구현

In [1]:
from sklearn.tree import DecisionTreeRegressor

In [5]:
class SimpleGradientBoostingClassifier:
    def __init__(self, n_estimators=100, learning_rate = 0.1, max_depth=3):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.initial_log_odds = 0       # 초기 예측값
        self.trees = []                 # 내부적으로 가지고 있는 estimator 모음 배열
    
    
    def log_odds(self, p): # p: 확률값
        # 확률값 -> 로짓변환 (컴큐터 더 편하게 할라고)
        return np.log(p / (1-p)) 
    
    
    def sigmoid(self, z):
        # z값 -> 0~1 사이의 확률 값 변환
        return 1 /(1+np.exp(-z))
    
    
    def fit(self, X, y):
        y_mean = np.mean(y) # 초기 예측값: 평균 (0과 1 다 더한 평균)
        self.initial_log_odds = self.log_odds(y_mean)
        y_pred_log_odds = np.full_like(y, self.initial_log_odds, dtype=np.float64)
        
        for _ in range(self.n_estimators): 
            y_pred_proba  = self.sigmoid(y_pred_log_odds)
            residual = y - y_pred_proba # 이 잔차 값으로 학습을 진행함
            tree = DecisionTreeRegressor(max_depth=self.max_depth)
            tree.fit(X, residual)
            self.trees.append(tree)
            y_pred_log_odds += self.learning_rate * tree.predict(X)
    
    
    def predict(self, X):
        return (self.predict_proba(X) >= 0.5).astype(int) # boolean을 0 혹은 1로 변환
        
    
    def predict_proba(self, X):
        y_pred_log_odds = np.full((X.shape[0],), self.initial_log_odds) # 초기값 세팅
        
        for tree in self.trees:
            y_pred_log_odds += self.learning_rate * tree.predict(X)
        
        return self.sigmoid(y_pred_log_odds)

In [12]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

data = load_breast_cancer()

X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, random_state=0)

simple_gb_clf = SimpleGradientBoostingClassifier(
    n_estimators=500,
    learning_rate=0.1,
    max_depth=3
)

simple_gb_clf.fit(X_train, y_train)

y_pred_train = simple_gb_clf.predict(X_train)
y_pred_test = simple_gb_clf.predict(X_test)

print(f"학습 정확도: {accuracy_score(y_train, y_pred_train)}")
print(f"평가 정확도: {accuracy_score(y_test, y_pred_test)}")

학습 정확도: 1.0
평가 정확도: 0.965034965034965


## GradientBoosting (라이브러리 활용)

In [13]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

data = load_breast_cancer()
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, random_state=0)

gb_clf = GradientBoostingClassifier(
    n_estimators=101,
    learning_rate=0.01,
    max_depth=3
)

gb_clf.fit(X_train, y_train)

y_pred_train = gb_clf.predict(X_train)
y_pred_test = gb_clf.predict(X_test)

print(f"학습 정확도: {accuracy_score(y_train, y_pred_train)}")
print(f"평가 정확도: {accuracy_score(y_test, y_pred_test)}")

학습 정확도: 0.9882629107981221
평가 정확도: 0.958041958041958


### HistGradientBoosting

- 고성능 GradientBoostin 모델로, 대규모 데이터셋 처리에 적합
- Histogram 기반으로 256개의 구간으로 나누어 처리 병합하는 방식
- 결측치가 있어도 전처리 필요 없음
- LightGBM의 영향을 받아 만들어진 scikit-learn 모델

In [15]:
from sklearn.ensemble import HistGradientBoostingClassifier

hist_gb_clf = HistGradientBoostingClassifier(
    learning_rate=0.1,
    max_depth= 3,
    max_bins= 255, # 255개의 구간으로 나누어 처리 (1개는 결측치 전용)
    early_stopping=True, # 반복 중 '일정 횟수' 이상 성능 향상이 없으면 학습 종료
    n_iter_no_change=5 # Early stopping에서 '일정 횟수' (기본값 10)
)

hist_gb_clf.fit(X_train, y_train)

y_pred_train = hist_gb_clf.predict(X_train)
y_pred_test = hist_gb_clf.predict(X_test)

print(f"학습 정확도: {accuracy_score(y_train, y_pred_train)}")
print(f"평가 정확도: {accuracy_score(y_test, y_pred_test)}")

학습 정확도: 0.9882629107981221
평가 정확도: 0.972027972027972


In [ ]:
# permutation_importances == 특성 중요도 (내부 학습 중이 아니라 외부 검정을 통해 계산)
from sklearn.inspection import permutation_importance

result = permutation_importance(
    hist_gb_clf,    # 모델
    X_train,        # X 데이터
    y_train,        # y 데이터
    n_repeats=5,    # 중요도 측정을 위해 무작위하게 사용할 feature를 섞는 횟수, 떨어진 만큼의 평균와 표춘편차 이용해서 계산
    random_state=0
)

# importances_mean: 평균 중요도
# importances_std: 중요도 표준편차
# importances: 특성 중요도
result.importances # shape는 (특성 개수, n_repeats)

(30, 5)

## 회귀모델

`sklearn.datasets.load_diabetes` 데이터셋
- 당뇨병 환자의 진단 자료를 바탕으로 만들어진 회귀용 데이터셋

**데이터셋 설명:**
- **목적**: 당뇨병 진행 정도(1년 후)를 예측
- **데이터 수**: 442개의 샘플
- **특성 수**: 10개의 특성 (10개의 입력 변수)
- **타겟**: 연속형 값, 당뇨병의 1년 후 진행 상황을 나타냄

**특성 설명:**
데이터셋의 각 특성(피처)은 환자의 다양한 생체 정보

1. **age**: 나이 (Age)
2. **sex**: 성별 (Sex)
3. **bmi**: 체질량 지수 (Body Mass Index)
4. **bp**: 평균 혈압 (Average Blood Pressure)
5. **s1**: 혈청 내 TC (Total Cholesterol)
6. **s2**: 혈청 내 LDL (Low-Density Lipoproteins)
7. **s3**: 혈청 내 HDL (High-Density Lipoproteins)
8. **s4**: 혈청 내 TCH (Total Cholesterol / HDL)
9. **s5**: 혈청 내 LTG (Log of Serum Triglycerides)
10. **s6**: 혈당 수치 (Blood Sugar Level)

**데이터 구조:**
- **입력 데이터 (data)**: (442, 10) 크기의 NumPy 배열
- **타겟 데이터 (target)**: (442,) 크기의 NumPy 배열로, 각 샘플의 타겟 값(연속형 값)을 포함

In [22]:
from sklearn.datasets import load_diabetes

# 1. 데이터 로드
diabetes_data = load_diabetes()

In [23]:
# 2. 데이터 분리
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(diabetes_data.data, diabetes_data.target, random_state=42)

In [33]:
# 3. HistoGradientBoostingRegressor 모델 생성
from sklearn.ensemble import HistGradientBoostingRegressor

hist_gb_reg = HistGradientBoostingRegressor(
    max_iter=100,
    learning_rate=0.05,
    max_depth= 3,
    #max_bins= 255, # 255개의 구간으로 나누어 처리 (1개는 결측치 전용)
    #early_stopping=True, # 반복 중 '일정 횟수' 이상 성능 향상이 없으면 학습 종료
    #n_iter_no_change=10, # Early stopping에서 '일정 횟수' (기본값 10)
    random_state=0,
    l2_regularization=0.5,
    min_samples_leaf=5
)

In [ ]:
# 4~6. 학습 > 예측 > mse, r2_score 계산
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score

hist_gb_reg.fit(X_train, y_train)

y_pred_train = hist_gb_reg.predict(X_train)
y_pred_test = hist_gb_reg.predict(X_test)

print(f"학습 MSE: {mean_squared_error(y_train, y_pred_train)}, 평가 MSE: {mean_squared_error(y_test, y_pred_test)}")
print(f"학습 R2: {r2_score(y_train, y_pred_train)}, 평가 R2: {r2_score(y_test, y_pred_test)}")

학습 MSE: 1489.73643339862, 평가 MSE: 2919.435300032376
학습 R2: 0.753543596826666, 평가 R2: 0.4720435672290497


**Cross-validation (교차검증) 맛보기**

In [ ]:
# 교차검증
from sklearn.model_selection import GridSearchCV

hist_gb_reb = HistGradientBoostingRegressor(random_state=0)


param_grid = {
    'max_iter': [100,200,300],
    'max_depth': [1,3,5],
    'learning_rate': [0.01, 0.05, 0.1],
    'min_samples_leaf': [10, 20, 30],
    'l2_regularization': [0.0, 0.1, 1.0],
    'max_bins': [255, 127]
}

# 성능 최적화시키는 hyper-parmeter 값 찾아줌 (모든 조합 때려서)
grid_search = GridSearchCV(hist_gb_reb, param_grid, cv=3, scoring='neg_mean_squared_error') # mse는 작을수록 좋은거니까 앞에 '-' 붙혀서 제일 큰 조합 찾을 수 있게 함
grid_search.fit(X_train, y_train)

# param_grid로 전달해준 하이퍼파라미터 중, 최고 성능 내는 조합
grid_search.best_params_

{'l2_regularization': 1.0,
 'learning_rate': 0.05,
 'max_bins': 255,
 'max_depth': 1,
 'max_iter': 300,
 'min_samples_leaf': 30}

In [ ]:
# 최고 성능 내는 하이퍼파라미터로 조합으로 학습 됨 모델
best_hist_gb_reg = grid_search.best_estimator_ 

y_pred_train = hist_gb_reg.predict(X_train)
y_pred_test = hist_gb_reg.predict(X_test)

print(f"학습 MSE: {mean_squared_error(y_train, y_pred_train)}, 평가 MSE: {mean_squared_error(y_test, y_pred_test)}")
print(f"학습 R2: {r2_score(y_train, y_pred_train)}, 평가 R2: {r2_score(y_test, y_pred_test)}")

학습 MSE: 1489.73643339862, 평가 MSE: 2919.435300032376
학습 R2: 0.753543596826666, 평가 R2: 0.4720435672290497
